Minimizzare sia per il numero di gruppi da creare sia per il costo delle chiamate api, il cui costo è pari a 1 per ogni gruppo e pari a 1 ogni 100 ip, sempre con il vincolo che al massimo in un gruppo ci possono essere 1000 ip. Vresione pulita

In [11]:
import pulp
import math

In [12]:
res = {"query": "services.http.response.html_title: \"X Acceleration Codec\" or services.http.response.body: \"X Acceleration Codec\"","field": "location.country","total": 2194,"duration": 1099,"total_omitted": 0,"potential_deviation": 0,"buckets": [{"key": "Netherlands","count": 454},{"key": "Spain","count": 392},{"key": "Germany","count": 267},{"key": "France","count": 246},{"key": "United States","count": 139},{"key": "Brazil","count": 108},{"key": "United Kingdom","count": 59},{"key": "Finland","count": 55},{"key": "Bulgaria","count": 41},{"key": "Italy","count": 39},{"key": "Romania","count": 32},{"key": "Ukraine","count": 32},{"key": "Canada","count": 25},{"key": "Serbia","count": 23},{"key": "Bosnia and Herzegovina","count": 21},{"key": "Thailand","count": 19},{"key": "India","count": 18},{"key": "Turkey","count": 16},{"key": "Singapore","count": 15},{"key": "Colombia","count": 14},{"key": "Pakistan","count": 14},{"key": "Poland","count": 14},{"key": "Lithuania","count": 12},{"key": "Israel","count": 10},{"key": "Portugal","count": 10},{"key": "Iraq","count": 9},{"key": "Russia","count": 8},{"key": "Croatia","count": 7},{"key": "Sweden","count": 7},{"key": "Austria","count": 6},{"key": "Switzerland","count": 6},{"key": "Indonesia","count": 5},{"key": "Belgium","count": 4},{"key": "Costa Rica","count": 4},{"key": "Palestinian Territory","count": 4},{"key": "Albania","count": 3},{"key": "Australia","count": 3},{"key": "Czech Republic","count": 3},{"key": "Estonia","count": 3},{"key": "Macedonia","count": 3},{"key": "Peru","count": 3},{"key": "South Africa","count": 3},{"key": "Tunisia","count": 3},{"key": "Argentina","count": 2},{"key": "Cyprus","count": 2},{"key": "Dominican Republic","count": 2},{"key": "Hungary","count": 2},{"key": "Iceland","count": 2},{"key": "Ireland","count": 2},{"key": "Paraguay","count": 2},{"key": "United Arab Emirates","count": 2},{"key": "Armenia","count": 1},{"key": "Bolivia","count": 1},{"key": "Chile","count": 1},{"key": "Denmark","count": 1},{"key": "Ecuador","count": 1},{"key": "Egypt","count": 1},{"key": "El Salvador","count": 1},{"key": "French Polynesia","count": 1},{"key": "Honduras","count": 1},{"key": "Japan","count": 1},{"key": "Jordan","count": 1},{"key": "Malta","count": 1},{"key": "Montenegro","count": 1},{"key": "Morocco","count": 1},{"key": "Myanmar","count": 1},{"key": "Nepal","count": 1},{"key": "Norway","count": 1},{"key": "Slovakia","count": 1},{"key": "Vietnam","count": 1}]}

In [13]:
data = res['buckets']

In [14]:
len(data)

70

In [15]:
# Maximum capacity per group
capacity = 1000

# Prepare data
countries = [item['key'] for item in data]
counts = [item['count'] for item in data]
num_items = len(countries)
num_bins = num_items  # Upper bound on the number of groups

In [16]:
# Creazione del problema di ottimizzazione
prob = pulp.LpProblem("Minimizza il Costo Totale", pulp.LpMinimize)

# Variabili di decisione

# Variabile binaria x[(i, j)]: indica se il paese i è assegnato al gruppo j
x = pulp.LpVariable.dicts('x',
    ((i, j) for i in range(num_items) for j in range(num_bins)),
    cat='Binary')

# Variabile binaria y[j]: indica se il gruppo j è utilizzato
y = pulp.LpVariable.dicts('y',
    (j for j in range(num_bins)),
    cat='Binary')

# Variabile continua s[j]: totale degli indirizzi IP nel gruppo j
s = pulp.LpVariable.dicts('s',
    (j for j in range(num_bins)),
    lowBound=0,
    cat='Continuous')

# Variabile intera c[j]: numero di blocchi da 100 IP nel gruppo j
c = pulp.LpVariable.dicts('c',
    (j for j in range(num_bins)),
    lowBound=0,
    cat='Integer')

# Funzione obiettivo: minimizzare il costo totale
# Costo totale = costo per gruppo (1 credito per gruppo) + costo per IP (1 credito ogni 100 IP)
prob += pulp.lpSum(y[j] + c[j] for j in range(num_bins))

# Vincoli

# 1. Vincolo di assegnazione: ogni paese deve essere assegnato esattamente a un gruppo
for i in range(num_items):
    prob += pulp.lpSum(x[(i, j)] for j in range(num_bins)) == 1

# 2. Vincolo di capacità del gruppo
for j in range(num_bins):
    # Calcolo del totale di IP nel gruppo j
    prob += s[j] == pulp.lpSum(counts[i] * x[(i, j)] for i in range(num_items))
    # Il totale di IP nel gruppo j non deve superare la capacità massima se il gruppo è utilizzato
    prob += s[j] <= capacity * y[j]

# 3. Relazione tra x e y: se un paese è assegnato a un gruppo, il gruppo deve essere utilizzato
for i in range(num_items):
    for j in range(num_bins):
        prob += x[(i, j)] <= y[j]

# 4. Vincoli per il calcolo del costo in base agli IP
for j in range(num_bins):
    # Assicura che s[j] sia al massimo 100 * c[j]
    prob += s[j] <= 100 * c[j]
    # Vincolo modificato per gestire correttamente c[j] quando y[j] = 0
    prob += s[j] >= 100 * (c[j] - y[j]) + 1


# 5. Vincolo di consistenza per gruppi non utilizzati
for j in range(num_bins):
    # Se il gruppo j non è utilizzato (y[j] = 0), allora c[j] deve essere 0
    prob += c[j] <= (capacity // 100) * y[j]

# Risoluzione del problema
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/6a6d42f42e414cdeb53ec604f128bed3-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/6a6d42f42e414cdeb53ec604f128bed3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5325 COLUMNS
At line 35846 RHS
At line 41167 BOUNDS
At line 46208 ENDATA
Problem MODEL has 5320 rows, 5110 columns and 20300 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 24.134 - 0.22 seconds
Cgl0003I 70 fixed, 70 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 140 rows, 4970 columns (4970 integer (4900 of which binary)) and 9870 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I 70 integer variables out of 4970 objects (4970 integer) have cost of 1 - high priority
Cbc0045I branch

1

In [17]:
print("Status:", pulp.LpStatus[prob.status])

# Raccolta dei gruppi utilizzati
bins = {}
for j in range(num_bins):
    if y[j].varValue > 0.5:
        bin_items = []
        total_count = 0
        for i in range(num_items):
            if x[(i, j)].varValue > 0.5:
                bin_items.append((countries[i], counts[i]))
                total_count += counts[i]
        if total_count > 0:
            group_cost = 1  # Costo per gruppo
            ip_cost = c[j].varValue  # Costo per 100 IP
            total_group_cost = group_cost + ip_cost
            bins[j] = {
                'items': bin_items,
                'total_count': total_count,
                'group_cost': group_cost,
                'ip_cost': ip_cost,
                'total_group_cost': total_group_cost
            }

# Calcolo del costo totale
total_cost = sum(bin_info['total_group_cost'] for bin_info in bins.values())
total_groups = len(bins)

# Output dei risultati
print(f"Numero totale di gruppi individuati: {total_groups}")
print(f"Costo totale (numero di crediti spesi): {total_cost}")
print("=" * 50)

group_number = 1
for bin_index, bin_info in bins.items():
    print(f"Gruppo {group_number}: Totale = {bin_info['total_count']} indirizzi IP | Costo gruppo = {bin_info['group_cost']} credito | Costo IP = {bin_info['ip_cost']} crediti | Costo totale gruppo = {bin_info['total_group_cost']} crediti")
    for country, count in bin_info['items']:
        print(f" - {country}: {count}")
    print("-" * 50)
    group_number += 1

Status: Optimal
Numero totale di gruppi individuati: 9
Costo totale (numero di crediti spesi): 31.0
Gruppo 1: Totale = 300 indirizzi IP | Costo gruppo = 1 credito | Costo IP = 3.0 crediti | Costo totale gruppo = 4.0 crediti
 - France: 246
 - Thailand: 19
 - India: 18
 - Israel: 10
 - Hungary: 2
 - Ireland: 2
 - Armenia: 1
 - French Polynesia: 1
 - Vietnam: 1
--------------------------------------------------
Gruppo 2: Totale = 100 indirizzi IP | Costo gruppo = 1 credito | Costo IP = 1.0 crediti | Costo totale gruppo = 2.0 crediti
 - Bulgaria: 41
 - Romania: 32
 - Iraq: 9
 - Russia: 8
 - Czech Republic: 3
 - Tunisia: 3
 - United Arab Emirates: 2
 - Chile: 1
 - Honduras: 1
--------------------------------------------------
Gruppo 3: Totale = 200 indirizzi IP | Costo gruppo = 1 credito | Costo IP = 2.0 crediti | Costo totale gruppo = 3.0 crediti
 - Brazil: 108
 - Italy: 39
 - Pakistan: 14
 - Lithuania: 12
 - Portugal: 10
 - Austria: 6
 - Switzerland: 6
 - Estonia: 3
 - Argentina: 2
------